# **Introduction:**

This file serves to design and test a custom implementation of an Adaptive Neuro-Fuzzy Inference System (ANFIS) within Keras. It is hoped that this custom function is able to be used like a standard Keras model, and will be trained and evaluated against the ANN designed previously. 

This ANFIS takes the load history, the distance to the task, and the total distance travelled thus far and performs inference about the suitability of a given robot for a task at hand. 

**Date Created:** 13/01/2025

**Date Modified:** 14/01/2025

# **Import Packages:** 

This section imports all the necessary packages for the ANFIS implementation. 

In [140]:
# import packages:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Sequential
from tensorflow.keras.optimizers import Adam

# **Layer Function & Class Definitions:**

Need to define the membership function to be used, as well as the custom layers of an ANFIS for implementation within Keras.

In [150]:
# define a function for triangular membership functions:
def triangular_mf(x, a, b, c):
    return np.maximum(0.0, np.minimum((x - a) / (b - a), (c - x) / (c - b)))

# custom fuzzification layer:
class FuzzificationLayer:
    def __init__(self, num_inputs, num_mfs):
        self.num_inputs  = num_inputs
        self.num_mfs = num_mfs

        # randomly initialize the parameters:
        self.params = np.random.uniform(low = -1.0, high = 1.0, size = (num_inputs, num_mfs, 3))

    def call(self, inputs):
        # ensure that inputs are numpy array
        inputs = np.array(inputs)

        mfs_list = []

        for i in range(self.num_inputs):
            mfs_for_feature = []
            for j in range(self.num_mfs):
                # apply the MF to each input feature
                mfs_for_feature.append(triangular_mf(inputs[:, i], *self.mf_params[i, j]))
            mfs_list.append(np.stack(mfs_for_feature, axis=1))  # stack the MFs for each input
        
        # concatenate all the MFs to form the output
        return np.concatenate(mfs_list, axis=1)
    
# custom firing strength layer:
class FiringStrengthLayer:
    def __init__(self, num_rules):
        self.num_rules = num_rules

    def call(self, inputs):
        return np.prod(inputs, axis = 1)
    
# custom normalization layer:
class NormalizationLayer:
    def __init__(self):
        pass

    def call(self, firing_strengths):
        total_strength = np.sum(firing_strengths, axis = 1, keepdims = True)
        return firing_strengths / total_strength
    
# custom rule consequent layer:
class ConsequentLayer:
    def __init__(self, num_rules, num_inputs):
            self.num_rules = num_rules
            self.num_inputs = num_inputs

            # initialize parameters for the consequents (weights and bias for each rule)
            self.params = np.random.uniform(low=-1.0, high=1.0, size=(num_rules, num_inputs + 1))

    def call(self, inputs):
        # inputs: (batch_size, num_rules), where each column is the normalized firing strength for a rule
        return np.dot(inputs, self.params.T)  # shape: (batch_size, num_rules)
    
# custom output layer:
class OutputLayer:
    def __init__(self):
        pass

    def call(self, rule_consequents, firing_strengths):
        # weighted sum of the rule consequents
        return np.sum(rule_consequents * firing_strengths[:, None], axis=1)


In [153]:
fl = FuzzificationLayer(num_inputs = 3, num_mfs = 3)
print('layer made')

layer made


# **Define Testing Parameters:**

This section defines the testing parameters, such as the number of inputs, the number of membership functions, and the expected number of rules. This section also defines a function for creating and compiling Keras models using the custom layers.

Testing Parameters:

In [142]:
# define parameters:
num_inputs = 3
num_mfs = 3
num_rules = num_mfs ** num_inputs

Model Generation Function:

In [143]:
# function to make models:
def make_model(num_inputs, num_mfs, num_rules, rate):

    # instantiate model:
    model = Sequential()

    # add fuzzification layer:
    model.add(FuzzificationLayer(num_inputs = num_inputs, num_mfs = num_mfs))

    # add firing strength layer:
    model.add(FiringStrengthLayer())

    # add normalization layer:
    model.add(NormalizationLayer())

    # add rule consequent layer:
    model.add(ConsequentLayer(num_rules = num_rules, num_inputs = num_inputs))

    # add output layer:
    model.add(OutputLayer())
    
    # return model:
    return model

# **Define Hybrid Training Function:**

It is important to note that training an ANFIS involves the use of a hybrid training algorithm usually, where the antecedent parameters (membership function parameters) are updated using back propagation through gradient descent, whereas the consequent parameters (rule consequent parameters) are updated using least-square optimization. This isn't natively supported within Keras, so it must be defined as follows.

Functions that are used in the hybrid training function:

In [144]:
# define function to compute the forward pass loss (mse):
def compute_loss(model, x_batch, y_batch):
    y_pred = model(x_batch)
    loss = tf.reduce_mean(tf.square(y_pred - y_batch))
    return loss

# define function to compute validation loss:
def compute_val_loss(model, val_dataset):
    val_loss = 0
    num_batches = 0
    for x_batch_val, y_batch_val in val_dataset:
        loss = compute_loss(model, x_batch_val, y_batch_val)
        val_loss += loss.numpy()
        num_batches += 1
    return val_loss / num_batches

# define function to update the consequent parameters:
def update_consequents(firing_strengths, targets):
    x = firing_strengths
    y = targets

    # least squares solution:
    x_t = tf.transpose(x)
    inverse_term = tf.linalg.inv(tf.matmul(x_t, x))
    params = tf.matmul(inverse_term, tf.matmul(x_t, y))

    return params

Hybrid training function:

In [145]:
# # define the function:
# def hybrid_train(model, train_dataset, val_dataset, epochs, rate, patience = 5):
#     optimizer = Adam(learning_rate = rate)  # define optimizer 
#     best_val_loss = np.inf                  # initialize best validation loss
#     no_improvement_count = 0

#     # for every epoch:
#     for epoch in range(epochs):
#         print(f'epoch {epoch + 1}/{epoch}') # update user

#         # training phase:
#         for x_batch_train, y_batch_train in train_dataset:
#             with tf.GradientTape() as tape:
#                 # forward pass:
#                 fuzzified_inputs = model.layers[0](x_batch_train)
#                 firing_strengths = model.layers[1](fuzzified_inputs)
#                 normalized_strengths = model.layers[2](firing_strengths)
#                 # consequent_outputs = model.layers[3](normalized_strengths)

#                 loss = compute_loss(model, x_batch_train, y_batch_train)
            
#         # backpropagation - need to compute the gradients of the antecedent parameters:
#         grads = tape.gradient(loss, model.layers[0].trainable_variables)
#         optimizer.apply_gradients(zip(grads, model.layers[0].trainable_variables))

#         # now do least squares estimation:
#         firing_strengths_reshaped = tf.reshape(firing_strengths, (-1, firing_strengths.shape[-1]))
#         consequent_params = update_consequents(firing_strengths_reshaped, y_batch_train)
#         model.layers[3].params_assign(consequent_params)

#         # validation phase
#         val_loss = compute_val_loss(model, val_dataset)
#         print(f"validation Loss: {val_loss:.4f}")

#         # early stopping
#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             no_improvement_count = 0  # reset counter if there's improvement
#         else:
#             no_improvement_count += 1
#             if no_improvement_count >= patience:
#                 print("early stopping triggered due to no improvement in validation loss.")
#                 break

#     print("training complete.")

# **Loading of Training Data:**

Need to load the data that had been generated using the FIS, such that it may be used to train the ANFIS model:

In [146]:
# load data from the CSV:
df = pd.read_csv('V3_Data.csv')

# extract the X and Y components of the dataframe:
x_data = df.drop(['Suitability'], axis = 1)
y_data = df['Suitability']

# need to split the data into training, validation, and testing:
x_train, x_temp, y_train, y_temp = train_test_split(x_data, y_data, test_size = 0.2)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size = 0.5)

# get split results:
print(f"there are {x_train.shape[0]} training examples")
print(f"there are {x_val.shape[0]} validation examples")
print(f"there are {x_test.shape[0]} testing examples")

there are 8000 training examples
there are 1000 validation examples
there are 1000 testing examples


Now need to perform normalization to improve model performance:

In [147]:
# normalize the training data:
x_min = x_train.min(axis = 0)
x_max = x_train.max(axis = 0)
x_train = (x_train - x_min) / (x_max - x_min)

# apply the normalization to the validation and testing data:
x_val = (x_val - x_min) / (x_max - x_min)
x_test = (x_test - x_min) / (x_max - x_min)

Convert to TensorFlow tensors:

In [148]:
# tensor conversion:
x_train_tensor = tf.convert_to_tensor(x_train, dtype = tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype = tf.float32)
x_val_tensor = tf.convert_to_tensor(x_val, dtype = tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val, dtype = tf.float32)
x_test_tensor = tf.convert_to_tensor(x_test, dtype = tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype = tf.float32)

# batch the data:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_tensor, y_train_tensor)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val_tensor, y_val_tensor)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_tensor, y_test_tensor)).batch(batch_size)

# **Train the Model:**

Use the hybrid training model to train the model:

In [149]:
# first need to instantiate a model:
model = make_model(num_inputs = num_inputs, num_mfs = num_mfs, num_rules = num_rules, rate = 0.001)

# train the model using the hybrid algorithm:
hybrid_train(model, train_dataset, val_dataset, 50, 0.001, patience = 5)

epoch 1/0


OperatorNotAllowedInGraphError: Exception encountered when calling FuzzificationLayer.call().

[1mCould not automatically infer the output shape / dtype of 'fuzzification_layer_17' (of type FuzzificationLayer). Either the `FuzzificationLayer.call()` method is incorrect, or you need to implement the `FuzzificationLayer.compute_output_spec() / compute_output_shape()` method. Error encountered:

Iterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.[0m

Arguments receive